In [1]:
# basic packages
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
import os
os.chdir('/content/drive/MyDrive/dacon/daconcup/')
import utils # is_holiday 변수

# packages for models
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib


# 1 
# 전체 프로세스 

- 최근 데이터만 가지고 학습시킨 모델




## preprocessing
---

- 변수 

    1. 이전 2주동안의 y들 -> 14*4=56

    2. y들의 통계량 -> 분산, 평균 -> 2*4=8

    3. is_holiday -> 1

    4. 대회 참가자 수 -> 1

    → 총 변수 66


In [2]:
data = pd.read_csv('/content/drive/MyDrive/dacon/daconcup/Data/train.csv', encoding='cp949')
info_cpt = pd.read_csv('/content/drive/MyDrive/dacon/daconcup/Data/info_competition.csv', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/dacon/daconcup/Data/submission.csv', encoding='cp949')

In [3]:
def info_cpt_preprocess(data):
    
    # 날짜는 날짜형태로 변경
    data['period_start'] = pd.to_datetime(data['period_start'])
    data['period_end'] = pd.to_datetime(data['period_end'])
    data['merge_deadline'] = pd.to_datetime(data['merge_deadline'])

    # 날짜별 시행 대회 수
    min_date = min(data['period_start'])
    max_date = max(data['period_end'])
    cnt_cpt_df = pd.DataFrame()
    cnt_cpt_df['date'] = pd.date_range(min_date, max_date)
    for idx, row in data.iterrows():
        date_interval = pd.date_range(row['period_start'].date(), row['period_end'].date())
        col_name = row['name']
        
        cnt_cpt_df[col_name] = 0
        cnt_cpt_df.loc[cnt_cpt_df['date'].apply(lambda x: x in date_interval), col_name] = row['participants']

    cnt_cpt_df['total_participants'] = np.sum(cnt_cpt_df.iloc[:, 1:], axis=1)

    return cnt_cpt_df # date | name(대회이름)s | total_participants
    



def train_preprocess(data, cnt_cpt_df):

    # 일별 데이터 생성
    data['date'] = pd.to_datetime(data['DateTime'].apply(lambda x: x[:10]))
    df = data.groupby('date')[['사용자', '세션', '신규방문자', '페이지뷰']].sum()
    y_cols = ['y_user', 'y_sess', 'y_new', 'y_views']
    df.columns = y_cols
    df = df.reset_index()
    # print(df.info())

    # 공휴일 여부 추가
    df = utils.add_isHoliday_column(df)


    # lag1~14변수 생성
    for col in y_cols:
        for i in range(1, 15): 
            col_name = col[2:] + '_lag' + str(i)
            df[col_name] = df[col].shift(i)

    # lag 변수들의 분산, 평균, 중앙값
    for col in y_cols:
        factor = col[2:]
        new_cols = [factor+'_lag_mean', factor+'_lag_std']
        df[new_cols] = df[[col for col in df if col.startswith(factor)]].apply(pd.DataFrame.describe, axis=1)[['mean', 'std']]

    # 대회 참가자 수
    df = df.merge(cnt_cpt_df.loc[:, ['date', 'total_participants']], on='date', how='left')

    return df

In [4]:
data_raw = data.copy()

cnt_cpt_df = info_cpt_preprocess(info_cpt)
data = train_preprocess(data_raw, cnt_cpt_df)
data.dropna(inplace=True)
data = data.reset_index(drop=True)
print('data shape:', data.shape)

data shape: (778, 71)


## Modeling


1. 특징

- 모델 변경할 때 바뀌는 요인
    - RandomForestRegressor의 hyperparameters
    - 모델 저장할 때 이름

- 각 y별로 모델 4개씩 생성

- walk-forward validation 사용
    - 5덩어리로 나눠서 마지막 7일씩 예측

2. 모델 종류

- rf_reg_1 : RandomForestRegressor(min_samples_leaf=3, bootstrap=False, random_state=42)

- rf_reg_2 : RandomForestRegressor(min_samples_leaf=3, max_features='sqrt', bootstrap=False, random_state=42) - select!


In [96]:
# 1) RandomForestRegressor(min_samples_leaf=3, bootstrap=False, random_state=42)
# 5-folds cross validation 
    ## 5덩어리로 나눠서 마지막 7일만 예측

def rf_reg_cv(data, y_name):
    X = data.iloc[:, 5:]
    y = data[y_name]

    errors = []
    
    for i in range(1, 6):
        start_idx = (data.shape[0] // 5) * (i-1) - 7 # test로 썼던 데이터를 train으로 사용
        last_idx = (data.shape[0] // 5) * i 
        
        if last_idx > data.shape[0]:
            last_idx = data.shape[0]

        if start_idx < 0:
            start_idx = 0
        train_idx = list(x for x in range(start_idx, last_idx-7))
        test_idx = list(x for x in range(last_idx-7, last_idx))

        X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
        y_train, y_test = y[train_idx], y[test_idx]    
        
        rf_reg = RandomForestRegressor(min_samples_leaf=3, bootstrap=False, random_state=42)
        rf_reg.fit(X_train, y_train)
        y_pred = rf_reg.predict(X_test)

        error = mean_squared_error(y_test, y_pred, squared=False) # RMSE는 False                                                                  
        errors.append(error)

    print('Modeling Complete for ' + y_name[2:])

    path = '/content/drive/MyDrive/dacon/daconcup/models/rf_reg_'+y_name[2:]+'1.pkl'
    joblib.dump(rf_reg, path)

    print(f'Model 저장 - {path}')
    print(f'validation error for {y_name[2:]}: {errors}')
    print(f'평균: {np.mean(errors)}, 표준편차: {np.std(errors)}')
    print('\n')

    return errors, rf_reg

In [97]:
# 1) RandomForestRegressor(min_samples_leaf=3, bootstrap=False, random_state=42)
errors_user, rf_reg_user = rf_reg_cv(data, 'y_user')
errors_sess, rf_reg_sess = rf_reg_cv(data, 'y_sess')
errors_new, rf_reg_new = rf_reg_cv(data, 'y_new')
errors_views, rf_reg_views = rf_reg_cv(data, 'y_views')

Modeling Complete for user
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_user1.pkl
validation error for user: [350.9551103163236, 129.79300922849157, 438.6387990496806, 387.7294406633869, 1359.2939698552443]
평균: 533.2820658226253, 표준편차: 426.259338369311


Modeling Complete for sess
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_sess1.pkl
validation error for sess: [400.1375991009664, 82.39653481011378, 439.78474848498325, 392.10898235130423, 1271.7526777933683]
평균: 517.2361085081471, 표준편차: 398.43400456158844


Modeling Complete for new
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_new1.pkl
validation error for new: [9.45403978004636, 83.78306300457507, 147.35661079618149, 179.9259471981262, 411.4568924869287]
평균: 166.39531065317155, 표준편차: 135.6879324577128


Modeling Complete for views
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_views1.pkl
validation error for views: [1256.805563626885, 2328.5025093121562, 5904.459872

In [98]:
# 2) RandomForestRegressor(min_samples_leaf=3, max_features='sqrt', bootstrap=False, random_state=42) - select!
# 5-folds cross validation 
    ## 5덩어리로 나눠서 마지막 7일만 예측

def rf_reg_cv(data, y_name):
    X = data.iloc[:, 5:]
    y = data[y_name]

    errors = []
    
    for i in range(1, 6):
        start_idx = (data.shape[0] // 5) * (i-1) - 7 # test로 썼던 데이터를 train으로 사용
        last_idx = (data.shape[0] // 5) * i 
        
        if last_idx > data.shape[0]:
            last_idx = data.shape[0]

        if start_idx < 0:
            start_idx = 0
        train_idx = list(x for x in range(start_idx, last_idx-7))
        test_idx = list(x for x in range(last_idx-7, last_idx))

        X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
        y_train, y_test = y[train_idx], y[test_idx]    
        
        rf_reg = RandomForestRegressor(min_samples_leaf=3, max_features='sqrt', bootstrap=False, random_state=42)
        rf_reg.fit(X_train, y_train)
        y_pred = rf_reg.predict(X_test)

        error = mean_squared_error(y_test, y_pred, squared=False) # RMSE는 False                                                                  
        errors.append(error)

    print('Modeling Complete for ' + y_name[2:])

    path = '/content/drive/MyDrive/dacon/daconcup/models/rf_reg_'+y_name[2:]+'2.pkl'
    joblib.dump(rf_reg, path)

    print(f'Model 저장 - {path}')
    print(f'validation error for {y_name[2:]}: {errors}')
    print(f'평균: {np.mean(errors)}, 표준편차: {np.std(errors)}')
    print('\n')

    return errors, rf_reg

In [99]:
# 2) RandomForestRegressor(min_samples_leaf=3, max_features='sqrt', bootstrap=False, random_state=42) - select!
errors_user, rf_reg_user = rf_reg_cv(data, 'y_user')
errors_sess, rf_reg_sess = rf_reg_cv(data, 'y_sess')
errors_new, rf_reg_new = rf_reg_cv(data, 'y_new')
errors_views, rf_reg_views = rf_reg_cv(data, 'y_views')

Modeling Complete for user
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_user2.pkl
validation error for user: [175.99209746572413, 93.72669681854718, 390.98460459038836, 573.8162962976618, 1053.539140896676]
평균: 457.61176721379945, 표준편차: 341.9924235795267


Modeling Complete for sess
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_sess2.pkl
validation error for sess: [204.13145284576953, 88.04168069431752, 390.4195609085891, 555.1549945600777, 1045.7900597915004]
평균: 456.7075497600508, 표준편차: 334.9069746138541


Modeling Complete for new
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_new2.pkl
validation error for new: [117.4000840428906, 53.60354819327477, 107.24516818020287, 143.9299562382413, 351.97256752009315]
평균: 154.83026483494055, 표준편차: 102.85534721668677


Modeling Complete for views
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_views2.pkl
validation error for views: [864.1962075040489, 1932.9283037319801, 5385.357

In [100]:
x, y =  zip(*sorted(zip(rf_reg_user.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_user feature_importance_ by rf_reg"
)
fig.show();

In [101]:
x, y =  zip(*sorted(zip(rf_reg_sess.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_sess feature_importance_ by rf_reg"
)
fig.show();

In [102]:
x, y =  zip(*sorted(zip(rf_reg_new.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_new feature_importance_ by rf_reg"
)
fig.show();

In [103]:
x, y =  zip(*sorted(zip(rf_reg_views.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_views feature_importance_ by rf_reg"
)
fig.show();

### 랜덤포레스트 모델
- rf_reg_user : 사용자
- rf_reg_sess : 세션
- rf_reg_new : 신규 방문자
- rf_reg_views : 페이지뷰


## 예측

- 선택된 모델 로드 맞게 하는지 확인

In [104]:
# 위에서 선택된 모델 로드
rf_reg_user = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_user2.pkl')
rf_reg_sess = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_sess2.pkl')
rf_reg_new = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_new2.pkl')
rf_reg_views = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_views2.pkl')

In [105]:
# prediction
    # 하나씩 예측하면서 예측한 결과로 또 예측하게 만들기
submission = pd.read_csv('/content/drive/MyDrive/dacon/daconcup/Data/submission.csv', encoding='cp949', parse_dates=['DateTime'])
submission_raw = submission.copy()

submission.columns = ['date', 'y_user', 'y_sess', 'y_new', 'y_views']
df_for_sub = pd.concat([data, submission], axis=0)

# holiday, 대회참가자수 변수 추가
del df_for_sub['isHoliday'], df_for_sub['total_participants'] 
df_for_sub = utils.add_isHoliday_column(df_for_sub)
df_for_sub = df_for_sub.merge(cnt_cpt_df[['date', 'total_participants']], how='left', on='date')

In [106]:
# 2020-11-09부터 2021-01-08까지 돌면서 1) x값 생성 2) prediction하고 그 값을 y에 채워넣기
import datetime

for day in pd.date_range('2020-11-09', '2021-01-08').astype(str).tolist():

    cols = ['user', 'sess', 'new', 'views']
    
    val_lst = []
    for i in cols:
        yname = 'y_' + i
        
    # lag 변수에 값 추가
        for j in range(1, 15):
            lag_day = datetime.datetime.strptime(day, '%Y-%m-%d') - datetime.timedelta(j)
            val_lst.append(df_for_sub.loc[df_for_sub['date']==lag_day, yname].values[0])
    # print(lag_day, val_lst)

    # lag 변수의 평균, 표준편차
    for i in range(4):
        start_idx = i * 14
        last_idx = (i+1) * 14

        mean = np.mean(val_lst[start_idx:last_idx])
        std = np.std(val_lst[start_idx:last_idx])
        
        val_lst += [mean, std]
    
    df_for_sub.loc[df_for_sub['date']==day, df_for_sub.columns[5:-2]] = val_lst

    X = df_for_sub[df_for_sub['date']==day].iloc[:, 5:].values
    y_pred_user = rf_reg_user.predict(X)[0]
    y_pred_sess = rf_reg_sess.predict(X)[0]
    y_pred_new = rf_reg_new.predict(X)[0]
    y_pred_views = rf_reg_views.predict(X)[0]
    
    print(f"{day}의 pred값 : {y_pred_user}, {y_pred_sess}, {y_pred_new}, {y_pred_views}")
    # 예측한 값을 현재의 y열에 추가
    df_for_sub.loc[df_for_sub['date']==day, df_for_sub.columns[1:5]] = [y_pred_user, y_pred_sess, y_pred_new, y_pred_views]
    display(df_for_sub.loc[df_for_sub['date']==day, :])

2020-11-09의 pred값 : 2094.673666666667, 2079.301333333333, 498.4263333333332, 57544.70616666666


,date,y_user,y_sess,y_new,y_views,user_lag1,user_lag2,user_lag3,user_lag4,user_lag5,user_lag6,user_lag7,user_lag8,user_lag9,user_lag10,user_lag11,user_lag12,user_lag13,user_lag14,sess_lag1,sess_lag2,sess_lag3,sess_lag4,sess_lag5,sess_lag6,sess_lag7,sess_lag8,sess_lag9,sess_lag10,sess_lag11,sess_lag12,sess_lag13,sess_lag14,new_lag1,new_lag2,new_lag3,new_lag4,new_lag5,new_lag6,new_lag7,new_lag8,new_lag9,new_lag10,new_lag11,new_lag12,new_lag13,new_lag14,views_lag1,views_lag2,views_lag3,views_lag4,views_lag5,views_lag6,views_lag7,views_lag8,views_lag9,views_lag10,views_lag11,views_lag12,views_lag13,views_lag14,user_lag_mean,user_lag_std,sess_lag_mean,sess_lag_std,new_lag_mean,new_lag_std,views_lag_mean,views_lag_std,isHoliday,total_participants
778,2020-11-09,2094.673667,2079.301333,498.426333,57544.706167,2492.0,2472.0,3663.0,4155.0,4516.0,5418.0,4758.0,1995.0,2006.0,3240.0,3954.0,3413.0,3332.0,3220.0,2420.0,2417.0,3576.0,4037.0,4472.0,5304.0,4717.0,1977.0,2023.0,3154.0,3946.0,3340.0,3262.0,3108.0,522.0,531.0,825.0,1044.0,1196.0,1524.0,1213.0,422.0,447.0,716.0,922.0,810.0,817.0,774.0,50486.0,57386.0,88015.0,102901.0,112683.0,150888.0,120423.0,47970.0,51312.0,90175.0,107834.0,87497.0,77518.0,79405.0,3473.857143,989.133377,3410.928571,973.148386,840.214286,308.130811,87463.785714,28919.175055,0,3513


2020-11-10의 pred값 : 2437.9563333333335, 2414.1114999999995, 574.6621666666668, 60431.51316666667


,date,y_user,y_sess,y_new,y_views,user_lag1,user_lag2,user_lag3,user_lag4,user_lag5,user_lag6,user_lag7,user_lag8,user_lag9,user_lag10,user_lag11,user_lag12,user_lag13,user_lag14,sess_lag1,sess_lag2,sess_lag3,sess_lag4,sess_lag5,sess_lag6,sess_lag7,sess_lag8,sess_lag9,sess_lag10,sess_lag11,sess_lag12,sess_lag13,sess_lag14,new_lag1,new_lag2,new_lag3,new_lag4,new_lag5,new_lag6,new_lag7,new_lag8,new_lag9,new_lag10,new_lag11,new_lag12,new_lag13,new_lag14,views_lag1,views_lag2,views_lag3,views_lag4,views_lag5,views_lag6,views_lag7,views_lag8,views_lag9,views_lag10,views_lag11,views_lag12,views_lag13,views_lag14,user_lag_mean,user_lag_std,sess_lag_mean,sess_lag_std,new_lag_mean,new_lag_std,views_lag_mean,views_lag_std,isHoliday,total_participants
779,2020-11-10,2437.956333,2414.1115,574.662167,60431.513167,2094.673667,2492.0,2472.0,3663.0,4155.0,4516.0,5418.0,4758.0,1995.0,2006.0,3240.0,3954.0,3413.0,3332.0,2079.301333,2420.0,2417.0,3576.0,4037.0,4472.0,5304.0,4717.0,1977.0,2023.0,3154.0,3946.0,3340.0,3262.0,498.426333,522.0,531.0,825.0,1044.0,1196.0,1524.0,1213.0,422.0,447.0,716.0,922.0,810.0,817.0,57544.706167,50486.0,57386.0,88015.0,102901.0,112683.0,150888.0,120423.0,47970.0,51312.0,90175.0,107834.0,87497.0,77518.0,3393.47669,1050.327741,3337.450095,1030.399614,820.530452,320.293915,85902.336155,29886.135283,0,3513


2020-11-11의 pred값 : 2522.7505, 2410.122999999999, 638.4430000000003, 62117.42116666666


...

,date,y_user,y_sess,y_new,y_views,user_lag1,user_lag2,user_lag3,user_lag4,user_lag5,user_lag6,user_lag7,user_lag8,user_lag9,user_lag10,user_lag11,user_lag12,user_lag13,user_lag14,sess_lag1,sess_lag2,sess_lag3,sess_lag4,sess_lag5,sess_lag6,sess_lag7,sess_lag8,sess_lag9,sess_lag10,sess_lag11,sess_lag12,sess_lag13,sess_lag14,new_lag1,new_lag2,new_lag3,new_lag4,new_lag5,new_lag6,new_lag7,new_lag8,new_lag9,new_lag10,new_lag11,new_lag12,new_lag13,new_lag14,views_lag1,views_lag2,views_lag3,views_lag4,views_lag5,views_lag6,views_lag7,views_lag8,views_lag9,views_lag10,views_lag11,views_lag12,views_lag13,views_lag14,user_lag_mean,user_lag_std,sess_lag_mean,sess_lag_std,new_lag_mean,new_lag_std,views_lag_mean,views_lag_std,isHoliday,total_participants
836,2021-01-06,1811.680833,1781.869333,456.085833,53139.477167,1795.024,1897.5205,1915.061333,1927.177333,2048.296167,2316.7835,2226.903333,1856.720833,1790.745833,1905.406,2003.457,1927.392833,2042.733833,2356.487333,1750.238833,1887.929333,1869.296167,1905.012333,1990.862167,2116.731,2141.478167,1834.114,1758.561667,1908.596167,1905.822333,1918.700833,2001.412333,2210.7605,448.820167,487.375,459.773667,459.526167,472.156833,543.175833,531.7855,451.886833,437.6165,482.235667,486.953833,460.685833,495.361667,558.4265,51189.172,50039.853833,49691.7265,50830.9185,52301.837833,56041.6485,55212.053667,51977.568667,51123.856167,49303.8495,50671.9045,50798.5035,53738.33,58898.9675,2000.69356,174.57193,1942.82256,131.7652,483.984286,35.623878,52272.870762,2655.959958,0,781


2021-01-07의 pred값 : 2222.412, 2150.7413333333325, 527.5798333333333, 56674.56516666666


,date,y_user,y_sess,y_new,y_views,user_lag1,user_lag2,user_lag3,user_lag4,user_lag5,user_lag6,user_lag7,user_lag8,user_lag9,user_lag10,user_lag11,user_lag12,user_lag13,user_lag14,sess_lag1,sess_lag2,sess_lag3,sess_lag4,sess_lag5,sess_lag6,sess_lag7,sess_lag8,sess_lag9,sess_lag10,sess_lag11,sess_lag12,sess_lag13,sess_lag14,new_lag1,new_lag2,new_lag3,new_lag4,new_lag5,new_lag6,new_lag7,new_lag8,new_lag9,new_lag10,new_lag11,new_lag12,new_lag13,new_lag14,views_lag1,views_lag2,views_lag3,views_lag4,views_lag5,views_lag6,views_lag7,views_lag8,views_lag9,views_lag10,views_lag11,views_lag12,views_lag13,views_lag14,user_lag_mean,user_lag_std,sess_lag_mean,sess_lag_std,new_lag_mean,new_lag_std,views_lag_mean,views_lag_std,isHoliday,total_participants
837,2021-01-07,2222.412,2150.741333,527.579833,56674.565167,1811.680833,1795.024,1897.5205,1915.061333,1927.177333,2048.296167,2316.7835,2226.903333,1856.720833,1790.745833,1905.406,2003.457,1927.392833,2042.733833,1781.869333,1750.238833,1887.929333,1869.296167,1905.012333,1990.862167,2116.731,2141.478167,1834.114,1758.561667,1908.596167,1905.822333,1918.700833,2001.412333,456.085833,448.820167,487.375,459.773667,459.526167,472.156833,543.175833,531.7855,451.886833,437.6165,482.235667,486.953833,460.685833,495.361667,53139.477167,51189.172,50039.853833,49691.7265,50830.9185,52301.837833,56041.6485,55212.053667,51977.568667,51123.856167,49303.8495,50671.9045,50798.5035,53738.33,1961.77881,149.902496,1912.187476,114.65633,476.674238,29.586936,51861.478595,1949.984719,0,781


2021-01-08의 pred값 : 2212.8995, 2085.1873333333324, 514.9911666666667, 54804.167


,date,y_user,y_sess,y_new,y_views,user_lag1,user_lag2,user_lag3,user_lag4,user_lag5,user_lag6,user_lag7,user_lag8,user_lag9,user_lag10,user_lag11,user_lag12,user_lag13,user_lag14,sess_lag1,sess_lag2,sess_lag3,sess_lag4,sess_lag5,sess_lag6,sess_lag7,sess_lag8,sess_lag9,sess_lag10,sess_lag11,sess_lag12,sess_lag13,sess_lag14,new_lag1,new_lag2,new_lag3,new_lag4,new_lag5,new_lag6,new_lag7,new_lag8,new_lag9,new_lag10,new_lag11,new_lag12,new_lag13,new_lag14,views_lag1,views_lag2,views_lag3,views_lag4,views_lag5,views_lag6,views_lag7,views_lag8,views_lag9,views_lag10,views_lag11,views_lag12,views_lag13,views_lag14,user_lag_mean,user_lag_std,sess_lag_mean,sess_lag_std,new_lag_mean,new_lag_std,views_lag_mean,views_lag_std,isHoliday,total_participants
838,2021-01-08,2212.8995,2085.187333,514.991167,54804.167,2222.412,1811.680833,1795.024,1897.5205,1915.061333,1927.177333,2048.296167,2316.7835,2226.903333,1856.720833,1790.745833,1905.406,2003.457,1927.392833,2150.741333,1781.869333,1750.238833,1887.929333,1869.296167,1905.012333,1990.862167,2116.731,2141.478167,1834.114,1758.561667,1908.596167,1905.822333,1918.700833,527.579833,456.085833,448.820167,487.375,459.773667,459.526167,472.156833,543.175833,531.7855,451.886833,437.6165,482.235667,486.953833,460.685833,56674.565167,53139.477167,51189.172,50039.853833,49691.7265,50830.9185,52301.837833,56041.6485,55212.053667,51977.568667,51123.856167,49303.8495,50671.9045,50798.5035,1974.612964,163.370862,1922.853833,128.563244,478.975536,32.097431,52071.209679,2271.902554,0,781


In [107]:
sub = df_for_sub.loc[df_for_sub['date'].apply(lambda x: x in pd.date_range('2020-11-09', '2021-01-08')), df_for_sub.columns[:5]].reset_index(drop=True)
sub.columns = submission_raw.columns 
sub

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,2094.673667,2079.301333,498.426333,57544.706167
1,2020-11-10,2437.956333,2414.111500,574.662167,60431.513167
2,2020-11-11,2522.750500,2410.123000,638.443000,62117.421167
3,2020-11-12,2538.512833,2415.068667,628.998833,64434.700500
4,2020-11-13,2474.891500,2420.233667,621.884000,67650.608500
...,...,...,...,...,...
56,2021-01-04,1897.520500,1887.929333,487.375000,50039.853833
57,2021-01-05,1795.024000,1750.238833,448.820167,51189.172000
58,2021-01-06,1811.680833,1781.869333,456.085833,53139.477167
59,2021-01-07,2222.412000,2150.741333,527.579833,56674.565167


In [108]:
sub.to_csv('/content/drive/MyDrive/dacon/daconcup/submission/03_randomforest_reg2.csv', index = False, encoding = 'euc-kr')

## 예측 시각화

In [110]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(shared_xaxes=True,rows=4, cols=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_user'], name='users'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_sess'], name='sessions'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_new'], name='new_users'), row=3, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_views'], name='page views'), row=4, col=1)

요일별 변동을 잡지 못 함. 요일을 새로운 변수로 추가해보자.

# 2
# 요일 변수를 추가한 새로운 모델 

- 이전 모델보다 성능이 더 떨어짐. 이미 isHoliday 변수가 있는데 7개의 더미 변수가 추가되면서 변수가 너무 많아져서인 듯.

In [6]:
# prediction
    # 하나씩 예측하면서 예측한 결과로 또 예측하게 만들기
submission = pd.read_csv('/content/drive/MyDrive/dacon/daconcup/Data/submission.csv', encoding='cp949', parse_dates=['DateTime'])
submission_raw = submission.copy()

submission.columns = ['date', 'y_user', 'y_sess', 'y_new', 'y_views']
df_for_sub = pd.concat([data, submission], axis=0)

# holiday, 대회참가자수 변수, 요일 변수(더미 변수로) 추가 
del df_for_sub['isHoliday'], df_for_sub['total_participants'] 
df_for_sub = utils.add_isHoliday_column(df_for_sub)
df_for_sub = df_for_sub.merge(cnt_cpt_df[['date', 'total_participants']], how='left', on='date')
df_for_sub['dayofweek'] = df_for_sub['date'].dt.dayofweek
df_for_sub = pd.concat([df_for_sub, pd.get_dummies(df_for_sub['dayofweek'], prefix='week')], axis=1)
del df_for_sub['dayofweek']

# 모델링 데이터
data = df_for_sub.dropna()

In [28]:
# 3) 요일을 추가한 모델

def rf_reg_cv(data, y_name):
    X = data.iloc[:, 5:]
    y = data[y_name]

    errors = []
    
    for i in range(1, 6):
        start_idx = (data.shape[0] // 5) * (i-1) - 7 # test로 썼던 데이터를 train으로 사용
        last_idx = (data.shape[0] // 5) * i 
        
        if last_idx > data.shape[0]:
            last_idx = data.shape[0]

        if start_idx < 0:
            start_idx = 0
        train_idx = list(x for x in range(start_idx, last_idx-7))
        test_idx = list(x for x in range(last_idx-7, last_idx))

        X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
        y_train, y_test = y[train_idx], y[test_idx]    
        
        rf_reg = RandomForestRegressor(n_estimators=200, min_samples_leaf=3, max_features='sqrt', bootstrap=False, random_state=42)
        rf_reg.fit(X_train, y_train)
        y_pred = rf_reg.predict(X_test)

        error = mean_squared_error(y_test, y_pred, squared=False) # RMSE는 False                                                                  
        errors.append(error)

    print('Modeling Complete for ' + y_name[2:])

    path = '/content/drive/MyDrive/dacon/daconcup/models/rf_reg_'+y_name[2:]+'3.pkl'
    joblib.dump(rf_reg, path)

    print(f'Model 저장 - {path}')
    print(f'validation error for {y_name[2:]}: {errors}')
    print(f'평균: {np.mean(errors)}, 표준편차: {np.std(errors)}')
    print('\n')

    return errors, rf_reg

In [29]:
# 3) RandomForestRegressor(min_samples_leaf=3, max_features='sqrt', bootstrap=False, random_state=42) + 요일
errors_user, rf_reg_user = rf_reg_cv(data, 'y_user')
errors_sess, rf_reg_sess = rf_reg_cv(data, 'y_sess')
errors_new, rf_reg_new = rf_reg_cv(data, 'y_new')
errors_views, rf_reg_views = rf_reg_cv(data, 'y_views')

Modeling Complete for user
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_user3.pkl
validation error for user: [168.22280147024958, 93.8568570280855, 385.6281587871091, 584.1437235695223, 1092.181532315955]
평균: 464.80661463418437, 표준편차: 357.6801760440362


Modeling Complete for sess
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_sess3.pkl
validation error for sess: [206.43355932329655, 87.36328025528402, 407.6878893214354, 541.3214154620106, 1021.891580274523]
평균: 452.9395449273099, 표준편차: 324.95436360516186


Modeling Complete for new
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_new3.pkl
validation error for new: [128.4502288747946, 57.31585849579609, 108.13409552800499, 137.9385641341388, 351.07093305717996]
평균: 156.58193601798288, 표준편차: 101.15916728214539


Modeling Complete for views
Model 저장 - /content/drive/MyDrive/dacon/daconcup/models/rf_reg_views3.pkl
validation error for views: [816.7362349596739, 1956.3692864838965, 5458.24829

## 모델별 변수 중요도 시각화

In [19]:
x, y =  zip(*sorted(zip(rf_reg_user.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_user feature_importance_ by rf_reg"
)
fig.show();

In [20]:
x, y =  zip(*sorted(zip(rf_reg_sess.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_sess feature_importance_ by rf_reg"
)
fig.show();

In [21]:
x, y =  zip(*sorted(zip(rf_reg_new.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_new feature_importance_ by rf_reg"
)
fig.show();

In [22]:
x, y =  zip(*sorted(zip(rf_reg_views.feature_importances_, data.columns[5:]), reverse=True)[:10])

trace = go.Bar(x=x[::-1], y=y[::-1],
               marker=dict(
                   color=x,
                   colorscale='Viridis',
               ),
               name='Random Forest Feature importance',
               orientation='h'
               )

fig = go.Figure(data=[trace])
fig.update_xaxes(range=[0,1])
fig.update_layout(
    width = 800,
    height = 500,
    title = "y_views feature_importance_ by rf_reg"
)
fig.show();

## 예측

In [23]:
# 위에서 선택된 모델 로드
rf_reg_user = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_user3.pkl')
rf_reg_sess = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_sess3.pkl')
rf_reg_new = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_new3.pkl')
rf_reg_views = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_views3.pkl')

In [25]:
# 2020-11-09부터 2021-01-08까지 돌면서 1) x값 생성 2) prediction하고 그 값을 y에 채워넣기

for day in pd.date_range('2020-11-09', '2021-01-08').astype(str).tolist():

    cols = ['user', 'sess', 'new', 'views']
    
    val_lst = []
    for i in cols:
        yname = 'y_' + i
        
    # lag 변수에 값 추가
        for j in range(1, 15):
            lag_day = datetime.datetime.strptime(day, '%Y-%m-%d') - datetime.timedelta(j)
            val_lst.append(df_for_sub.loc[df_for_sub['date']==lag_day, yname].values[0])
    # print(lag_day, val_lst)

    # lag 변수의 평균, 표준편차
    for i in range(4):
        start_idx = i * 14
        last_idx = (i+1) * 14

        mean = np.mean(val_lst[start_idx:last_idx])
        std = np.std(val_lst[start_idx:last_idx])
        
        val_lst += [mean, std]
    
    df_for_sub.loc[df_for_sub['date']==day, df_for_sub.columns[5:-9]] = val_lst

    X = df_for_sub[df_for_sub['date']==day].iloc[:, 5:].values
    y_pred_user = rf_reg_user.predict(X)[0]
    y_pred_sess = rf_reg_sess.predict(X)[0]
    y_pred_new = rf_reg_new.predict(X)[0]
    y_pred_views = rf_reg_views.predict(X)[0]
    
    # print(f"{day}의 pred값 : {y_pred_user}, {y_pred_sess}, {y_pred_new}, {y_pred_views}")
    # 예측한 값을 현재의 y열에 추가
    df_for_sub.loc[df_for_sub['date']==day, df_for_sub.columns[1:5]] = [y_pred_user, y_pred_sess, y_pred_new, y_pred_views]
    # display(df_for_sub.loc[df_for_sub['date']==day, :])

In [26]:
sub = df_for_sub.loc[df_for_sub['date'].apply(lambda x: x in pd.date_range('2020-11-09', '2021-01-08')), df_for_sub.columns[:5]].reset_index(drop=True)
sub.columns = submission_raw.columns 
sub

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,2879.528833,2793.570667,660.861333,74103.752167
1,2020-11-10,2837.760833,2801.952667,712.333667,71373.446167
2,2020-11-11,2894.859333,2902.979833,709.868000,73202.381167
3,2020-11-12,2835.216000,2750.443667,655.559167,73188.405333
4,2020-11-13,2761.723333,2664.995333,604.185000,72148.363667
...,...,...,...,...,...
56,2021-01-04,2841.888333,2670.811833,670.091667,71217.357833
57,2021-01-05,2821.566167,2672.135333,690.203500,71591.057500
58,2021-01-06,2817.903333,2681.121500,692.125000,71591.500167
59,2021-01-07,2833.772167,2690.546167,697.785833,71741.162500


In [27]:
# 예측 시각화
fig = make_subplots(shared_xaxes=True,rows=4, cols=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_user'], name='users'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_sess'], name='sessions'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_new'], name='new_users'), row=3, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_views'], name='page views'), row=4, col=1)

# 3

# 전체 데이터로 학습 

- 하루하루 변동성에 대한 예측률이 더 떨어짐

In [37]:
# 위에서 추가한 요일변수 제거
data = data.iloc[:, :-7]
df_for_sub = df_for_sub.iloc[:, :-7]

In [38]:
# 모델링
for y_name in data.columns[1:5]:
    X = data.iloc[:, 5:]
    y = data[y_name]
    
    rf_reg = RandomForestRegressor(min_samples_leaf=3, max_features='sqrt', bootstrap=False, random_state=42)
    rf_reg.fit(X, y)
    print(f"모델 스코어: {rf_reg.score(X, y)}")

    model_path = '/content/drive/MyDrive/dacon/daconcup/models/rf_reg_'+y_name[2:]+'_all.pkl'
    joblib.dump(rf_reg, model_path)
    print(f'Save model..! {model_path}')

모델 스코어: 0.9923339594168219
Save model..! /content/drive/MyDrive/dacon/daconcup/models/rf_reg_user_all.pkl
모델 스코어: 0.9914822592887028
Save model..! /content/drive/MyDrive/dacon/daconcup/models/rf_reg_sess_all.pkl
모델 스코어: 0.9714108141751449
Save model..! /content/drive/MyDrive/dacon/daconcup/models/rf_reg_new_all.pkl
모델 스코어: 0.9915381157157297
Save model..! /content/drive/MyDrive/dacon/daconcup/models/rf_reg_views_all.pkl


In [42]:
# 위에서 선택된 모델 로드
rf_reg_user = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_user_all.pkl')
rf_reg_sess = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_sess_all.pkl')
rf_reg_new = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_new_all.pkl')
rf_reg_views = joblib.load('/content/drive/MyDrive/dacon/daconcup/models/rf_reg_views_all.pkl')

# 2020-11-09부터 2021-01-08까지 돌면서 1) x값 생성 2) prediction하고 그 값을 y에 채워넣기

for day in pd.date_range('2020-11-09', '2021-01-08').astype(str).tolist():

    cols = ['user', 'sess', 'new', 'views']
    
    val_lst = []
    for i in cols:
        yname = 'y_' + i
        
    # lag 변수에 값 추가
        for j in range(1, 15):
            lag_day = datetime.datetime.strptime(day, '%Y-%m-%d') - datetime.timedelta(j)
            val_lst.append(df_for_sub.loc[df_for_sub['date']==lag_day, yname].values[0])
    # print(lag_day, val_lst)

    # lag 변수의 평균, 표준편차
    for i in range(4):
        start_idx = i * 14
        last_idx = (i+1) * 14

        mean = np.mean(val_lst[start_idx:last_idx])
        std = np.std(val_lst[start_idx:last_idx])
        
        val_lst += [mean, std]
    
    df_for_sub.loc[df_for_sub['date']==day, df_for_sub.columns[5:-2]] = val_lst

    X = df_for_sub[df_for_sub['date']==day].iloc[:, 5:].values
    y_pred_user = rf_reg_user.predict(X)[0]
    y_pred_sess = rf_reg_sess.predict(X)[0]
    y_pred_new = rf_reg_new.predict(X)[0]
    y_pred_views = rf_reg_views.predict(X)[0]
    
    # print(f"{day}의 pred값 : {y_pred_user}, {y_pred_sess}, {y_pred_new}, {y_pred_views}")
    # 예측한 값을 현재의 y열에 추가
    df_for_sub.loc[df_for_sub['date']==day, df_for_sub.columns[1:5]] = [y_pred_user, y_pred_sess, y_pred_new, y_pred_views]
    # display(df_for_sub.loc[df_for_sub['date']==day, :])

In [44]:
# 예측 시각화
fig = make_subplots(shared_xaxes=True,rows=4, cols=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_user'], name='users'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_sess'], name='sessions'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_new'], name='new_users'), row=3, col=1)
fig.add_trace(go.Scatter(x=df_for_sub['date'], y=df_for_sub['y_views'], name='page views'), row=4, col=1)

# RF 결론

- dacon은 계속 성장세에 있는 듯 한데 이 성장세를 설명할 수 있는 요인은 외부에서 찾아야 할 듯. 이 요인을 찾아야 함.
    - 데이터 분석 서치트렌드 or 언론 보도수 활용 가능 
    - 만약 내부에서 성장세를 찾는다면 기간으로 찾을 수 있을 듯
    - 또 다른 방법이 있을까?

- 트리기반 모델은 주어진 데이터의 y값 평균으로 결과를 예측해서 성장세에 있는 그래프를 예측하지 못 함. 정해진 y의 범위가 있음.

- 트리기반 모델 사용 비추

- 딥러닝 모델 사용하기

